<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/G125.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [38]:
clmns = [
  "Open 1", "Open 2", "Open 3", "Open 4", "Open 5", "Open 6", "Open 7", "Open 8", "Open 9", "Open 10", "Open 11", "Open 12", "Open 13", "Open 14", "Open 15", "Open 16", "Open 17", "Open 18", "Open 19", "Open 20", "Open 21", "Open 22", "Open 23", "Open 24", "Open 25", "Open 26", "Open 27", "Open 28", "Open 29", "Open 30", 
  "High 1", "High 2", "High 3", "High 4", "High 5", "High 6", "High 7", "High 8", "High 9", "High 10", "High 11", "High 12", "High 13", "High 14", "High 15", "High 16", "High 17", "High 18", "High 19", "High 20", "High 21", "High 22", "High 23", "High 24", "High 25", "High 26", "High 27", "High 28", "High 29", "High 30",
  "Low 1", "Low 2", "Low 3", "Low 4", "Low 5", "Low 6", "Low 7", "Low 8", "Low 9", "Low 10", "Low 11", "Low 12", "Low 13", "Low 14", "Low 15", "Low 16", "Low 17", "Low 18", "Low 19", "Low 20", "Low 21", "Low 22", "Low 23", "Low 24", "Low 25", "Low 26", "Low 27", "Low 28", "Low 29", "Low 30",
  "Close 1", "Close 2", "Close 3", "Close 4", "Close 5", "Close 6", "Close 7", "Close 8", "Close 9", "Close 10", "Close 11", "Close 12", "Close 13", "Close 14", "Close 15", "Close 16", "Close 17", "Close 18", "Close 19", "Close 20", "Close 21", "Close 22", "Close 23", "Close 24", "Close 25", "Close 26", "Close 27", "Close 28", "Close 29", "Close 30",
  "Volume 1", "Volume 2", "Volume 3", "Volume 4", "Volume 5", "Volume 6", "Volume 7", "Volume 8", "Volume 9", "Volume 10", "Volume 11", "Volume 12", "Volume 13", "Volume 14", "Volume 15", "Volume 16", "Volume 17", "Volume 18", "Volume 19", "Volume 20", "Volume 21", "Volume 22", "Volume 23", "Volume 24", "Volume 25", "Volume 26", "Volume 27", "Volume 28", "Volume 29", "Volume 30",
  "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10,10))
    row = scaler.fit_transform(row)
    return row
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 30:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print(" Null ")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [40]:
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(31, data.shape[0]-1):
        grow = []
        ggrow = []
        gggrow = []
        ggggrow = []
        gggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggrow.append([data[i-x][1] - data[i-(1+x)][1]] )
            gggrow.append([data[i-x][2] - data[i-(1+x)][2]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )
            gggggrow.append([data[i-x][-1] - data[i-(1+x)][-1]] )
        arr = np.array(grow).flatten()
        arr2 = np.array(ggrow).flatten()
        arr3 = np.array(gggrow).flatten()
        arr4 = np.array(ggggrow).flatten()
        arr5 = np.array(gggggrow).flatten()
        arr = scaler(arr.reshape(-1, 1))
        arr2 = scaler(arr2.reshape(-1, 1))
        arr3 = scaler(arr3.reshape(-1, 1))
        arr4 = scaler(arr4.reshape(-1, 1))
        arr5 = scaler(arr5.reshape(-1, 1))
        
        sugg = "sell"
        if data[i][3] > data[i-1][3]:
            sugg = "buy"

        arr = np.concatenate((arr, arr2, arr3, arr4,arr5), axis=0).reshape(-1,1)
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [ ]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1mo")

In [ ]:
folder_name = extract_data(1)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

In [ ]:
model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))

opt = tf.keras.optimizers.Adamax()
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xTrain,yTrain,epochs=1,batch_size=20000,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"MCG122_{acr}.h5")

In [35]:
def process_for_prediction(data,index):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        ggggrow = []
        gggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggrow.append([data[i-x][1] - data[i-(1+x)][1]] )
            gggrow.append([data[i-x][2] - data[i-(1+x)][2]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )
            gggggrow.append([data[i-x][-1] - data[i-(1+x)][-1]] )

        arr = np.array(grow).flatten()
        arr2 = np.array(ggrow).flatten()
        arr3 = np.array(gggrow).flatten()
        arr4 = np.array(ggggrow).flatten()
        arr5 = np.array(gggggrow).flatten()

        arr = scaler(arr.reshape(-1, 1))
        arr2 = scaler(arr2.reshape(-1, 1))
        arr3 = scaler(arr3.reshape(-1, 1))
        arr4 = scaler(arr4.reshape(-1, 1))
        arr5 = scaler(arr5.reshape(-1, 1))

        arr = np.concatenate((arr, arr2, arr3, arr4,arr5), axis=0).reshape(-1,1)
        return arr
def make_prediction_for_yf(symbol,period,timeframe,index):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe),index)
    return f"YF  : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction_for_tv(symbol,exchange,timeframe,tindex):
   tv = TvDatafeed()
   raw_data = process_for_prediction(tv.get_hist(symbol=symbol,exchange=exchange,interval=timeframe,n_bars=35),tindex)
   return  f"TVB : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex))

In [ ]:
make_prediction("btc-usd","max","1mo","btcusd","bitstamp",Interval.in_monthly,-1,-2)

In [ ]:
yf.download("btc-usd",period="max",interval="1mo")